In [1]:
import sqlite3
import numpy as np

In [181]:
conn = sqlite3.connect('../livs.db')  # Create db and establish connection
conn.row_factory = sqlite3.Row
curs = conn.cursor()
result = []
rows = curs.execute('select * from livs limit 4000')
for row in rows:
        result.append(row)

rows = np.array(result)
print (rows)

conn.close()

[['Talg nöt' '1' '656.3' ..., '0.0' '73.0'
  'Övrigt fett (ister, talg, kokosfett)']
 ['Späck gris' '2' '763.0' ..., '0.0' '3.0'
  'Övrigt fett (ister, talg, kokosfett)']
 ['Ister gris' '3' '884.3' ..., '0.0' '40.0'
  'Övrigt fett (ister, talg, kokosfett)']
 ..., 
 ['Havredryck oberikad' '5964' '40.6' ..., '' '155.0'
  'Vegetabiliska produkter och mjölkersättning']
 ['Kryddblandning taco' '5973' '372.6' ..., '' '7.0' 'Kryddor']
 ['Tortilla wrap' '5974' '313.7' ..., '' '39.0' 'Mjukt bröd']]


Jag har inte stött på den här kraftfulla syntaxen för np-arrayer, att man bara anger ett villkor rakt upp och ner och så returnerar den en array (en- eller multidimensionell) av celler som uppyller villkoret!

In [182]:
rows [1] == 0.0


/Users/organization/anaconda/envs/traffic_signs/lib/python3.5/site-packages/ipykernel_launcher.py:1: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  """Entry point for launching an IPython kernel.


False

numpy.where returnerar index för de celler som uppfyller villkoret.

In [183]:
empties = np.array(np.where(rows == ''))

numpy.where returnerar en tuple, men genom att göra om till en numpy-array kan man sen transponera och visa lite lättare

In [184]:
empties.T #.T = transpose

array([[   0,   39],
       [   0,   49],
       [   1,   15],
       ..., 
       [2085,   58],
       [2086,   58],
       [2087,   58]])

I empties[1] ligger alla förekomster av tomma celler per kolumn. En siffra i empties[1] betyder att det finns en tom i en cell i den kolumnen. I nästa summerar jag ihop vilka kolumner som har tomma värden, och hur många som finns. 

In [185]:
unique, counts = np.unique(empties[1], return_counts=True) #https://stackoverflow.com/questions/28663856/how-to-count-the-occurrence-of-certain-item-in-an-ndarray-in-python
columnsWithEmpties = np.array(list(zip(unique, counts)))
print(columnsWithEmpties)

[[  15 2026]
 [  34   83]
 [  36   86]
 [  39 1521]
 [  49 1268]
 [  58  502]
 [  60    7]]


Nu är det dags att ta reda på vilka kolumner det här är.

In [186]:
conn = sqlite3.connect('../livs.db')  # Create db and establish connection
conn.row_factory = sqlite3.Row
curs = conn.cursor()
columnObject = []

for row in conn.execute('PRAGMA table_info (livs);'):
    columnObject.append(row)

columns = np.array(columnObject)

conn.close()

In [187]:
print(columns)

[[0 'Livsmedelsnamn' 'TEXT' 0 None 0]
 [1 'Livsmedelsnummer' 'TEXT' 0 None 0]
 [2 'Energi_kcal' 'REAL' 0 None 0]
 [3 'Energi_kJ' 'REAL' 0 None 0]
 [4 'Kolhydrater_g' 'REAL' 0 None 0]
 [5 'Fett_g' 'REAL' 0 None 0]
 [6 'Protein_g' 'REAL' 0 None 0]
 [7 'Fibrer_g' 'REAL' 0 None 0]
 [8 'Vatten_g' 'REAL' 0 None 0]
 [9 'Alkohol_g' 'REAL' 0 None 0]
 [10 'Aska_g' 'REAL' 0 None 0]
 [11 'Monosackarider_g' 'REAL' 0 None 0]
 [12 'Disackarider_g' 'REAL' 0 None 0]
 [13 'Sackaros_g' 'REAL' 0 None 0]
 [14 'Fullkorn_totalt_g' 'REAL' 0 None 0]
 [15 'Sockerarter_g' 'REAL' 0 None 0]
 [16 'Summa_mattade_fettsyror_g' 'REAL' 0 None 0]
 [17 'Fettsyra_4_0_10_0_g' 'REAL' 0 None 0]
 [18 'Fettsyra_12_0_g' 'REAL' 0 None 0]
 [19 'Fettsyra_14_0_g' 'REAL' 0 None 0]
 [20 'Fettsyra_16_0_g' 'REAL' 0 None 0]
 [21 'Fettsyra_18_0_g' 'REAL' 0 None 0]
 [22 'Fettsyra_20_0_g' 'REAL' 0 None 0]
 [23 'Summa_enkelomattade_fettsyror_g' 'REAL' 0 None 0]
 [24 'Fettsyra_16_1_g' 'REAL' 0 None 0]
 [25 'Fettsyra_18_1_g' 'REAL' 0 None 0]
 

In [188]:
columns[([1 , 25])]

array([[1, 'Livsmedelsnummer', 'TEXT', 0, None, 0],
       [25, 'Fettsyra_18_1_g', 'REAL', 0, None, 0]], dtype=object)

In [189]:
columnsWithEmpties.T[0]

array([15, 34, 36, 39, 49, 58, 60])

In [190]:
columns[columnsWithEmpties.T[0]]

array([[15, 'Sockerarter_g', 'REAL', 0, None, 0],
       [34, 'Retinol_mikrog', 'REAL', 0, None, 0],
       [36, 'beta_Karoten_mikrog', 'REAL', 0, None, 0],
       [39, 'Vitamin_K_mikrog', 'REAL', 0, None, 0],
       [49, 'Jod_mikrog', 'REAL', 0, None, 0],
       [58, 'Avfall_skal_etc', 'REAL', 0, None, 0],
       [60, 'Huvudgrupp', 'TEXT', 0, None, 0]], dtype=object)

## Kolumnerna med få tomma

I några få fall, som Kolesterol och Vitamin A är det bara ett fåtal livsmedel som saknar värde. Vilka??

In [192]:
conn = sqlite3.connect('../livs.db')  # Create db and establish connection
conn.row_factory = sqlite3.Row
curs = conn.cursor()
result = []
rows = curs.execute('select * from livs where Huvudgrupp = "" limit 4000')
for row in rows:
        result.append(row)

rows = np.array(result)
print (len(rows), rows)
    
cols = [0,1,60]    

print (rows[:,cols])

conn.close()

7 [['Linser torkade' '884' '325.4' '1361.5' '48.5' '1.0' '24.0' '11.5' '12.0'
  '0.0' '3.0' '0.3' '2.1' '2.1' '0.0' '' '0.15' '0.0' '0.0' '0.0' '0.13'
  '0.01' '0.01' '0.18' '0.0' '0.17' '0.47' '0.38' '0.09' '0.0' '0.0' '0.0'
  '0.0' '0.0' '0.0' '5.0' '60.0' '0.0' '0.34' '' '0.37' '0.22' '0.0' '2.0'
  '6.0' '0.6' '0.0' '433.0' '375.0' '0.7' '6.8' '79.0' '790.0' '80.0'
  '30.0' '0.08' '11.0' '3.1' '0.0' '20.0' '']
 ['Linser torkade kokta u salt' '885' '104.8' '438.5' '15.6' '0.3' '7.8'
  '3.7' '71.7' '0.0' '0.9' '0.1' '0.7' '0.7' '0.0' '' '0.05' '0.0' '0.0'
  '0.0' '0.04' '0.0' '0.0' '0.05' '0.0' '0.05' '0.14' '0.11' '0.03' '0.0'
  '0.0' '0.0' '0.0' '0.0' '0.0' '1.0' '12.0' '0.0' '0.07' '' '0.07' '0.06'
  '0.0' '0.6' '1.9' '0.11' '0.0' '181.0' '119.0' '' '2.1' '25.0' '250.0'
  '25.0' '12.0' '0.03' '3.0' '1' '0.0' '43.0' '']
 ['Lax rå' '1239' '180.9' '756.8' '0.0' '12.0' '18.4' '0.0' '68.0' '0.0'
  '1.6' '0.0' '0.0' '0.0' '0.0' '' '2.55' '0.0' '0.0' '0.35' '1.72' '0.36'
  '0.0' '4.97' '1

Då är det väl bara att lägga in dem, rad efter rad. Hälleflundran hittar jag på nåt ställe som säger sig använda Livsmedelsverkets data. Flora sätter jag till 0.0 för det verkar alla margarinerna vara. Det betyder att även kolesterol går bra att ha med i klustringen.

Vad gäller A-vitamin är det två ris-sorter som saknar värden. Alla andra har 0.0, så det får bli 0.0 på dem också.

In [180]:
conn = sqlite3.connect('../livs.db')  # Create db and establish connection
conn.row_factory = sqlite3.Row
curs = conn.cursor()

sql = "UPDATE livs SET Vitamin_A_mikrog = 0.0 WHERE Livsmedelsnummer = '2516';"
print(sql)
curs.execute(sql) #Kommentera bort denna rad vid testning

conn.commit()
conn.close()

UPDATE livs SET Vitamin_A_mikrog = 0.0 WHERE Livsmedelsnummer = '2516';


## Fixa till databasen lite...

Det är även några celler i databasen som har None i stället för '' för tomma värden. Det är bättre att alla har samma struktur.

Det gäller Avfall_skal_etc som jag ändrade från '' till None för att jag trodde det skulle gå att klustra då, och Huvudgrupp, där det tydligen finns några livsmedel som inte tillhör någon kategori! (Intressant att kolla vilka...)

För att kunna använda kod från andra delar av projektet vill jag ha en array "new" med Livsmedelsnummer.

In [146]:
empties = np.array(np.where(rows.T[60] == None))
print(empties[0], len(empties[0]))

[ 735  736 1028 1034 1320 1621 1622] 7


In [147]:
change = rows[empties[0]].T[1:]
new = change[0]
new

array(['884', '885', '1239', '1245', '1561', '1910', '1911'], dtype=object)

In [149]:
conn = sqlite3.connect('../livs.db')  # Create db and establish connection
conn.row_factory = sqlite3.Row
curs = conn.cursor()

for row in new:
    print(row)
    sql = "UPDATE livs SET Huvudgrupp = '' WHERE Livsmedelsnummer = '"+row+"';"
    print(sql)
    #curs.execute(sql) #Kommentera bort denna rad vid testning

conn.commit()
conn.close()

884
UPDATE livs SET Huvudgrupp = '' WHERE Livsmedelsnummer = '884';
885
UPDATE livs SET Huvudgrupp = '' WHERE Livsmedelsnummer = '885';
1239
UPDATE livs SET Huvudgrupp = '' WHERE Livsmedelsnummer = '1239';
1245
UPDATE livs SET Huvudgrupp = '' WHERE Livsmedelsnummer = '1245';
1561
UPDATE livs SET Huvudgrupp = '' WHERE Livsmedelsnummer = '1561';
1910
UPDATE livs SET Huvudgrupp = '' WHERE Livsmedelsnummer = '1910';
1911
UPDATE livs SET Huvudgrupp = '' WHERE Livsmedelsnummer = '1911';
